In [ ]:
import requests, re, time, os, json, pickle, shutil, pdfplumber
from bs4 import BeautifulSoup
import pandas as pd
import seaborn as sns



In [ ]:
start = 2018

item_list = []



for year in range(start, 2025):
    print(year)
    url = f"https://www.ipma.pt/pt/publicacoes/boletins.jsp?cmbDep=sis&cmbTema=bsi&cmbAno={str(year)}&idDep=sis&idTema=bsi&curAno={str(year)}"
    #print(url)
    response = requests.get(url)
    stat = response.status_code
    soup = BeautifulSoup(response.content, 'html.parser')

    time.sleep(1)
    b_url="https://www.ipma.pt"
    items = []

    for i in soup.find_all("td", "class"=="apli_sat_img"):
        items.append(i.find("a"))

        items = list(filter(lambda items: items is not None, items))

        for j in range(0,len(items)):
            item_list.append(b_url+items[j]["href"])
            
item_list = [item for item in item_list if "bsi_mm_pm" in item]




In [ ]:
item_list[0].split('/')[-1].split(".")[0]

In [ ]:
## download


def download_file(url):
    local_filename = url.split('/')[-1]
    os.makedirs(local_filename.split(".")[0], exist_ok=True)
    with requests.get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename




for j in item_list:
    download_file(j)

In [ ]:
url1 = "C:/Users/Utilizador/Desktop/IRONHACK/FinalProj/Bol/"
url2 = "bc_2011-01_papel.pdf"
url_lib = "C:/texlive/2021/bin/win32/pdftotext.exe"

import os, subprocess

# Set the path to your PDF file
url1 = "C:/Users/Utilizador/Desktop/IRONHACK/FinalProj/Bol/"
url2 = "bc_2011-01_papel.pdf"

# Set the path to the pdftotext executable
pdftotext_path = f"{url_lib}"  # Adjust this path as necessary

SCRIPT_DIR = f"{url1}"
pdf_file_name = f"{url2}"
pdf_file_path = os.path.join(SCRIPT_DIR, pdf_file_name)

if not os.path.isfile(pdf_file_path):
    print(f"File {pdf_file_name} not found in {SCRIPT_DIR}.")
else:
    args = [pdftotext_path,
            '-enc', 'UTF-8',
            pdf_file_path, '-']

    res = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = res.stdout.decode('utf-8')

    print(output)


In [ ]:
my_list = output.replace("\n","    ").split("    ")
search_string = "Listagem de Hipocentros dos Sismos Próximos\r"

index = my_list.index(search_string)

display(" ".join(my_list[index]))

In [ ]:
my_list2 = my_list[index:]
my_list3 = "    ".join(my_list2)
my_list3.split("    ")

In [ ]:

pdf_file_path = "C:/Users/Utilizador/Desktop/IRONHACK/FinalProj/Bol/bc_2011-01_papel.pdf"
pattern = (r"\d\d-\d\d-\d\d\d\d \d\d:\d\d:\d\d.\d \d\d.\d\d\d.\S \d\d.\d\d\d")

page_list=[]
table = []

with pdfplumber.open(pdf_file_path) as pdf:
    for page in pdf.pages:
        # Extract text
        text = page.extract_text()
        page_list.append(text)
        
        for pag in page_list:
            paging = pag.split("\n")
            for p in paging:
                if re.match(pattern, p):
                    table.append(p)
        """
        # Extract tables
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                
               
                
        """


In [ ]:

# Specify the directory you want to scan
directory_path = "C:/Users/Utilizador/Desktop/IRONHACK/FinalProj/Bol/"  # Change this to your folder path

# Create a list of filenames
file_names = [file for file in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, file))]

table = []
for file in file_names:
    pdf_file_path = f"C:/Users/Utilizador/Desktop/IRONHACK/FinalProj/Bol/{file}"
    pattern = (r"\d\d-\d\d-\d\d\d\d \d\d:\d\d:\d\d.\d \d\d.\d\d\d.\S \d\d.\d\d\d")

    page_list=[]

    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            # Extract text
            text = page.extract_text()
            page_list.append(text)
            
    for pag in page_list:
        paging = pag.split("\n")
        for p in paging:
            if re.match(pattern, p):
                table.append(p)
        

In [ ]:
len(table)

In [189]:
table

NameError: name 'table' is not defined

In [ ]:
table2 = [line.split(",") for line in table]

In [ ]:
table3 = []

for i in range(len(table2)):    
    j = table2[i][0].replace(" ",",")
    table3.append(list(j))
    


In [ ]:
# Convert spaces to commas while keeping each element separate
converted_data = [item[0].replace(" ", ",").split(",") for item in table2]

# Process each row
processed_data = []

for row in converted_data:
    if 'ml' in row:
        ml_index = row.index('ml')  # Find the index of "ml"
        
        # Move the last element to the position after "ml"
        last_element = row.pop()  # Remove the last element
        row.insert(ml_index + 1, last_element)  # Insert it after "ml"
        
        # Join remaining elements into a single string
        joined_row = ' '.join(row)
        processed_data.append(joined_row)



In [ ]:
display(processed_data)

In [ ]:
import pandas as pd 

df = pd.DataFrame(processed_data, columns=["date","time", "lat","lon","depth",,"rms","int", "region"])

In [ ]:
df = pd.DataFrame(table2, columns=["index","data"])

In [ ]:
df.to_csv("backup_in_case.csv")

In [ ]:
df=pd.read_csv("backup_in_case_copy.csv")

In [ ]:
df = df.drop(columns=["Unnamed: 2"])

In [ ]:
df

In [ ]:
df2 = df.__deepcopy__()

In [ ]:
l1 = list(df["data"])
l1

In [ ]:
l2 = [s.split(",") for s in l1]

l2

In [ ]:
# Splitting each string into elements
#l3 = [s[0].split() for s in l2]
l3 = [s[0].replace(" ml", "ml").split() for s in l2]

In [ ]:
l3

In [ ]:
merged_results = []

for row in l3:
    # Look for elements that contain "ml"
    ml_found = False  # Flag to track if "ml" is found
    for i, element in enumerate(row):
        if "ml" in element:
            # Join all elements after the one that contains "ml"
            merged_string = ' '.join(row[i + 1:])  # Join elements after "ml"
            merged_results.append(merged_string)
            ml_found = True  # Set the flag to True
            break  # Exit the loop after finding the first "ml"
    
    if not ml_found:
        merged_results.append(" ")  # Append a space if "ml" is not found

In [ ]:
len(merged_results)

In [ ]:
# Pattern to match the numeric value at the end of the string
pattern = r'(\d\.\d)(.*)$'

# Initialize lists to hold extracted values
magnitudes = []
parameters = []


for entry in merged_results:
    match = re.search(pattern, entry)
    if match:
        magnitudes.append(match.group(1))  # The numeric value
        parameters.append(match.group(2).strip())  # Any parameters after the number
    else:
        magnitudes.append(" ")
        parameters.append(" ")



In [ ]:
l3[0]

In [ ]:
l3_first_six_columns = [row[:6] for row in l3]

In [ ]:
l3_df = pd.DataFrame(l3_first_six_columns, columns=["date","time", "lat","lon","depth","mag"])
display(l3_df.shape)
l3_df.head(5)

In [ ]:
rms_df = pd.DataFrame(magnitudes, columns=["Rms"])
display(rms_df.shape)
rms_df.head(5)


In [ ]:
int_df = pd.DataFrame(parameters, columns=["Int"])
display(int_df.shape)
int_df.head(5)

In [ ]:
together = pd.concat([l3_df, rms_df, int_df], axis=1)

In [ ]:
together.shape

In [ ]:
together.to_csv("semi-processed.csv")

In [180]:
together = pd.read_csv("semi-processed.csv", index_col="index")

# Convert 'date' to datetime
together['date'] = pd.to_datetime(together['date'], format='%d-%m-%Y')

# Convert 'time' to timedelta
together['time'] = pd.to_timedelta(together['time'])

# Combine 'date' and 'time' into a single datetime column
together['datetime'] = together['date'] + together['time']

In [181]:
together.describe()

,date,time,datetime
count,2556354,2556354,2556354
mean,2018-05-06 21:47:02.355589376,0 days 11:36:42.679560929,2018-05-07 09:23:45.035150848
min,2005-01-01 00:00:00,0 days 00:00:01.500000,2005-01-01 22:42:36.500000
25%,2014-12-17 00:00:00,0 days 04:44:53.100000,2014-12-17 20:04:22.200000
50%,2020-02-02 00:00:00,0 days 11:19:26.400000,2020-02-02 18:30:04.300000
75%,2022-02-26 00:00:00,0 days 18:31:04.500000,2022-02-26 05:41:27.400000
max,2023-10-31 00:00:00,0 days 23:59:58.200000,2023-10-31 23:29:24.500000
std,NaN,0 days 07:23:16.196861364,NaN


In [187]:
together["datetime"].unique()

<DatetimeArray>
['2011-01-01 00:57:28.600000', '2011-01-01 02:43:51.300000',
 '2011-01-01 06:50:57.800000', '2011-01-01 06:58:33.700000',
 '2011-01-01 07:08:21.100000', '2011-01-01 07:09:15.100000',
 '2011-01-01 09:01:36.800000', '2011-01-01 09:03:32.900000',
        '2011-01-01 09:13:00', '2011-01-01 09:14:02.600000',
 ...
 '2014-02-26 05:45:06.600000', '2014-02-26 09:48:58.900000',
 '2014-02-26 13:35:21.300000', '2014-02-27 02:44:31.600000',
 '2014-02-27 03:06:10.200000', '2014-02-27 04:03:29.800000',
 '2014-02-27 16:33:12.500000', '2014-02-28 00:07:00.500000',
 '2014-02-28 09:14:32.300000', '2014-02-28 10:53:15.500000']
Length: 36134, dtype: datetime64[ns]

In [188]:
together

,date,time,lat,lon,depth,mag,Rms,Int,datetime
index,,,,,,,,,
0,2011-01-01,0 days 00:57:28.600000,37.317°N,08.522°W,15,1.1ml,0.4,NaN,2011-01-01 00:57:28.600
1,2011-01-01,0 days 02:43:51.300000,38.669°N,08.486°W,0*,0.9ml,0.3,NaN,2011-01-01 02:43:51.300
2,2011-01-01,0 days 06:50:57.800000,36.538°N,07.663°W,31*,1.5ml,0.3,NaN,2011-01-01 06:50:57.800
3,2011-01-01,0 days 06:58:33.700000,36.355°N,09.698°W,31*,1.7ml,0.3,NaN,2011-01-01 06:58:33.700
4,2011-01-01,0 days 07:08:21.100000,37.036°N,04.850°W,0*,1.9ml,0.4,NaN,2011-01-01 07:08:21.100
...,...,...,...,...,...,...,...,...,...
2556349,2014-02-27,0 days 04:03:29.800000,37.333°N,08.511°W,12,0.9ml,0.3,NaN,2014-02-27 04:03:29.800
2556350,2014-02-27,0 days 16:33:12.500000,33.146°N,05.111°W,10*,2.2ml,0.4,NaN,2014-02-27 16:33:12.500
2556351,2014-02-28,0 days 00:07:00.500000,42.426°N,08.100°W,0*,1.5ml,0.3,NaN,2014-02-28 00:07:00.500
